<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/multistock_td3_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y protobuf tensorflow gym keras torch torchvision torchaudio || true
!pip install protobuf==3.20.3 tensorflow==2.18.0 stable-baselines3[extra] gymnasium gym-anytrading yfinance xgboost joblib matplotlib pandas numpy scipy scikit-learn imblearn
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


Found existing installation: protobuf 5.29.4
Uninstalling protobuf-5.29.4:
  Successfully uninstalled protobuf-5.29.4
Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: gym 0.25.2
Uninstalling gym-0.25.2:
  Successfully uninstalled gym-0.25.2
Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0
Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 2

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1

In [3]:
!rm -rf /content/drive

In [4]:
# === Mount Google Drive ===
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)
RESULTS_DIR = "/content/drive/MyDrive/Results_May_2025/results_td3_walkforward"
os.makedirs(RESULTS_DIR, exist_ok=True)

# === Imports ===
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import torch
from datetime import datetime
from stable_baselines3 import TD3
from stable_baselines3.common.vec_env import DummyVecEnv
from gymnasium.spaces import Box
from gym_anytrading.envs import StocksEnv

# === Date Ranges with Timezone Awareness ===
train_start = pd.to_datetime("2023-06-01").tz_localize("UTC")
train_end = pd.to_datetime("2024-06-01").tz_localize("UTC")
test_start = pd.to_datetime("2024-06-01").tz_localize("UTC")
test_end = pd.to_datetime("2025-04-28").tz_localize("UTC")

# === Stock List ===
TICKERS = [
    'AAPL', 'TSLA', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'BRK-B', 'JPM', 'JNJ',
    'XOM', 'V', 'PG', 'UNH', 'MA', 'HD', 'LLY', 'MRK', 'PEP', 'KO',
    'BAC', 'ABBV', 'AVGO', 'PFE', 'COST', 'CSCO', 'TMO', 'ABT', 'ACN', 'WMT',
    'MCD', 'ADBE', 'DHR', 'CRM', 'NKE', 'INTC', 'QCOM', 'NEE', 'AMD', 'TXN',
    'AMGN', 'UPS', 'LIN', 'PM', 'UNP', 'BMY', 'LOW', 'RTX', 'CVX', 'IBM',
    'GE', 'SBUX', 'ORCL'
]

# === Helper Functions ===
def clean_stock_data(df):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df.rename(columns=lambda x: x.capitalize(), inplace=True)
    df.rename(columns={'Adj Close': 'Close'}, inplace=True)
    df['Datetime'] = pd.to_datetime(df['Datetime'], errors='coerce')
    df.dropna(subset=['Datetime', 'Close'], inplace=True)
    df['Datetime'] = df['Datetime'].dt.tz_convert("UTC")
    df.sort_values(by='Datetime', inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

def compute_technical_indicators(df):
    df['ATR'] = (df['High'] - df['Low']).rolling(window=14).mean()
    df['EMA_10'] = df['Close'].ewm(span=10).mean()
    df['EMA_50'] = df['Close'].ewm(span=50).mean()
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    macd_fast = df['Close'].ewm(span=12).mean()
    macd_slow = df['Close'].ewm(span=26).mean()
    df['MACD_Line'] = macd_fast - macd_slow
    df['MACD_Signal'] = df['MACD_Line'].ewm(span=9).mean()
    df['MACD_Hist'] = df['MACD_Line'] - df['MACD_Signal']
    delta = df['Close'].diff()
    gain = delta.clip(lower=0).rolling(window=14).mean()
    loss = -delta.clip(upper=0).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    df['Volume_Avg'] = df['Volume'].rolling(window=20).mean()
    df['ADX'] = abs(df['High'] - df['Low']).rolling(window=14).mean()
    df.dropna(inplace=True)
    return df

class ContinuousTradingEnv(StocksEnv):
    def __init__(self, df, frame_bound, window_size):
        super().__init__(df=df, frame_bound=frame_bound, window_size=window_size)
        self.action_space = Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)

    def step(self, action):
        if isinstance(action, np.ndarray):
            action = action[0]
        if action < -0.3:
            discrete_action = 0
        elif action > 0.3:
            discrete_action = 1
        else:
            discrete_action = 2
        return super().step(discrete_action)

# === Training Loop ===
results = []

for symbol in TICKERS:
    print(f"\n Processing {symbol}")
    df = yf.download(symbol, period="720d", interval="1h", progress=False)
    if df is None or df.empty:
        print(f" No data for {symbol}")
        continue
    df.reset_index(inplace=True)
    df = clean_stock_data(df)
    df = compute_technical_indicators(df)

    df_train = df[(df['Datetime'] >= train_start) & (df['Datetime'] < train_end)].copy()
    df_test = df[(df['Datetime'] >= test_start) & (df['Datetime'] <= test_end)].copy()

    if len(df_train) < 300 or len(df_test) < 100:
        print(f" Skipping {symbol}: insufficient data.")
        continue

    env_train = DummyVecEnv([lambda: ContinuousTradingEnv(df_train, frame_bound=(50, len(df_train)), window_size=10)])
    env_test = DummyVecEnv([lambda: ContinuousTradingEnv(df_test, frame_bound=(50, len(df_test)), window_size=10)])

    model = TD3(
        "MlpPolicy",
        env_train,
        learning_rate=1e-4,
        buffer_size=100000,
        batch_size=256,
        gamma=0.995,
        policy_delay=2,
        verbose=0,
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )

    model.learn(total_timesteps=20000)

    obs = env_test.reset()
    trade_log, buy_price = [], None
    portfolio = []
    balance = 100000
    position = 0
    close_prices = df_test['Close'].values

    for i in range(len(df_test) - 50):
        action, _ = model.predict(obs)
        obs, _, done, _ = env_test.step(action)[:4]
        a = float(np.squeeze(action))
        price = close_prices[i + 50]

        if a > 0.3 and buy_price is None:
            buy_price = price
            position = balance / price
            balance = 0
            trade_log.append("BUY")
        elif a < -0.3 and buy_price is not None:
            balance = position * price
            position = 0
            buy_price = None
            trade_log.append("SELL")
        else:
            trade_log.append("HOLD")

        portfolio_value = balance if balance > 0 else position * price
        portfolio.append(portfolio_value)
        if done[0]:
            break

    final_value = portfolio[-1]
    cumulative_return = ((final_value / 100000) - 1) * 100
    daily_returns = pd.Series(portfolio).pct_change().fillna(0)
    sharpe = (daily_returns.mean() / (daily_returns.std() + 1e-6)) * np.sqrt(252)
    drawdown = ((pd.Series(portfolio).cummax() - pd.Series(portfolio)) / pd.Series(portfolio).cummax()).max() * 100

    results.append({
        "Ticker": symbol,
        "Sharpe": round(sharpe, 4),
        "Drawdown": round(drawdown, 2),
        "Return": round(cumulative_return, 2),
        "Final_Portfolio": round(final_value, 2)
    })

# === Save Results ===
results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(RESULTS_DIR, "td3_datebased_walkforward_summary.csv"), index=False)
print("Results saved to Google Drive")


Mounted at /content/drive

 Processing AAPL
YF.download() has changed argument auto_adjust default to True

 Processing TSLA

 Processing MSFT

 Processing GOOGL

 Processing AMZN

 Processing NVDA

 Processing META

 Processing BRK-B

 Processing JPM

 Processing JNJ

 Processing XOM

 Processing V

 Processing PG

 Processing UNH

 Processing MA

 Processing HD

 Processing LLY

 Processing MRK

 Processing PEP

 Processing KO

 Processing BAC

 Processing ABBV

 Processing AVGO

 Processing PFE

 Processing COST

 Processing CSCO

 Processing TMO

 Processing ABT

 Processing ACN

 Processing WMT

 Processing MCD

 Processing ADBE

 Processing DHR

 Processing CRM

 Processing NKE

 Processing INTC

 Processing QCOM

 Processing NEE

 Processing AMD

 Processing TXN

 Processing AMGN

 Processing UPS

 Processing LIN

 Processing PM

 Processing UNP

 Processing BMY

 Processing LOW

 Processing RTX

 Processing CVX

 Processing IBM

 Processing GE

 Processing SBUX

 Processing ORC

In [7]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display

# === Config ===
MODEL_NAME = "TD3"
DRIVE_DIR = "/content/drive/MyDrive/Results_May_2025/results_td3_walkforward"
os.makedirs(DRIVE_DIR + "/plots", exist_ok=True)
os.makedirs(DRIVE_DIR + "/data", exist_ok=True)

# === Save model_selector_ready_td3.csv ===
results_df["Model"] = MODEL_NAME
if "Accuracy" not in results_df.columns:
    results_df["Accuracy"] = np.nan

selector_df = results_df[["Ticker", "Model", "Sharpe", "Accuracy", "Drawdown", "Return", "Final_Portfolio"]]
selector_df.to_csv(f"{DRIVE_DIR}/model_selector_ready_td3.csv", index=False)
print("Saved model selector file")

Saved model selector file
